In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
hidden_size = 4

In [3]:
lstm = nn.LSTM(3, hidden_size)  # Input dim is 3, output dim is 4
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

Here:

* `sequence_length` = 5
* `batch_size` = 1
* `vocab_size` = 3

Effectively a 5 x 1 x 3 Tensor.

In [4]:
# hidden_size
hidden = (torch.randn(1, 1, hidden_size),
          torch.randn(1, 1, hidden_size))

`hidden` and `cell`: 

* `hidden_size` = 3
* `batch_size` = 1

In [5]:
inputs[0].shape

torch.Size([1, 3])

In [6]:
inputs[0].view(1, 1, -1).shape

torch.Size([1, 1, 3])

* `sequence_length` = 1
* `batch_size` = 1
* `vocab_size` = 3

In [7]:
out, hidden = lstm(i.view(1, 1, -1), hidden)

NameError: name 'i' is not defined

In [91]:
out.shape

torch.Size([1, 1, 4])

In [92]:
print(hidden[0].shape)
print(hidden[1].shape)

torch.Size([1, 1, 4])
torch.Size([1, 1, 4])


In [101]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print(inputs.shape)
hidden, cell = (torch.randn(1, 1, hidden_size), torch.randn(1, 1, hidden_size))  # clean out hidden state
print(hidden.shape)
print(cell.shape)
out, (hidden, cell) = lstm(inputs, (hidden, cell))
print(out)
print(hidden)
print(cell)

torch.Size([5, 1, 3])
torch.Size([1, 1, 4])
torch.Size([1, 1, 4])
tensor([[[-0.2790,  0.3213, -0.2033,  0.5494]],

        [[-0.0437,  0.1451, -0.1750,  0.1836]],

        [[ 0.0648,  0.1652, -0.1322, -0.0261]],

        [[ 0.3015,  0.1689, -0.0110,  0.1361]],

        [[ 0.3249,  0.1469,  0.0899,  0.0992]]], grad_fn=<StackBackward>)
tensor([[[0.3249, 0.1469, 0.0899, 0.0992]]], grad_fn=<StackBackward>)
tensor([[[0.4820, 0.3709, 0.1726, 0.1814]]], grad_fn=<StackBackward>)


In [82]:
out.shape

torch.Size([5, 1, 4])

`nn.LSTM` always takes in 3D tensors:

* First dimension: sequence length
* Second dimension

### Character model

In [47]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [48]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [49]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [53]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([0, 1, 2, 3, 4])
tensor([[-1.1192, -1.2072, -0.9823],
        [-1.0968, -1.2253, -0.9879],
        [-1.1267, -1.2088, -0.9745],
        [-1.1217, -1.1948, -0.9902],
        [-1.0156, -1.2633, -1.0354]])


In [56]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()


In [55]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.0858, -2.9355, -3.5374],
        [-5.2313, -0.0234, -4.0314],
        [-3.9098, -4.1279, -0.0368],
        [-0.0187, -4.7809, -4.5960],
        [-5.8170, -0.0183, -4.1879]])


Note: if output is $x$ for `NLL` loss, the probability is $e^x$.

## Full implementation

[here](https://github.com/L1aoXingyu/pytorch-beginner/blob/master/05-Recurrent%20Neural%20Network/recurrent_network.py)